In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd 

2022-12-02 11:06:04.177339: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 11:06:05.020257: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-02 11:06:05.227657: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-02 11:06:05.227697: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

# FIX PATHS!

In [2]:
##FIX PATHS TO LOAD CSVs
legit = pd.read_csv("legit_text.csv")

In [3]:
scam = pd.read_csv("scam_text.csv")

In [4]:
legit.drop(columns='Unnamed: 0', inplace=True)

In [5]:
legit.head()

,url,text
0,http://www.theaccessbankukltd.co.uk,menuaboutpersonalbusinessprivatedubainewsconta...
1,http://www.adambank.com,transfer contact usloginon 3 september 2022 we...
2,http://www.adib.co.uk,sign in​homeabout adibour brandmission objec...
3,http://www.aldermore.co.uk,log inpersonalbusinessintermediariesabout usco...
4,http://www.allfunds.com/en,cookie configurationallfunds bank s a u allf...


In [6]:
scam.drop(columns='Unnamed: 0', inplace=True)

In [7]:
scam.head()

,url,text
0,https://www.awesomeaussieshepherd.com,australian shepherd homeabout usavailable pup...
1,http://www.gclservice.co.za,index of \tname\tlast modified\tsize\tdescri...
2,https://www.gcloanservice.com,menuhomeloan applicationcontact usfaqsterms of...
3,http://www.authenicbiodocs.com,skip to contentpay with bitcoin25 discount fo...
4,https://www.thaiproductsllc.com,skip to content 61 3 9028 2716world wide shipp...


In [8]:
scam['Target'] = 1
legit['Target'] = 0

In [9]:
df = pd.concat([scam, legit])

In [10]:
df.tail()

,url,text,Target
2107,http://shop.hasbro.com/,skip to main contentnl nederlandsontdek spee...,0
2108,http://www.ropertech.com/,skip to content↵enterskip to contentsimple ide...,0
2109,http://www.arkocorp.com/,investorsoverviewnews eventsoverviewpress re...,0
2110,http://frontier.com/,skip to contentopen menufrontier communication...,0
2111,http://www.ameren.com/,skip to main contenttoggle navigationtoggle ac...,0


In [11]:
df.to_csv('full_text')

In [12]:
#Clean all text data 
from nltk.corpus import stopwords 
import string
from nltk import word_tokenize 
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dimitris/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
def clean(text):
    text = text.split()
    words_only = [word for word in text if word.isalpha()]
    for punctuation in string.punctuation:
        words_only = [word.replace(punctuation, ' ').lower() for word in words_only] # Remove Punctuation
    # lowercased = text.lower() # Lower Case
    #tokenized = word_tokenize(lowercased) # Tokenize
    # words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    # lemma=WordNetLemmatizer() # Initiate Lemmatizer
    # lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return without_stopwords

In [14]:
df['str_text'] = df['text'].astype('str')
df['clean_text'] = df['str_text'].apply(clean)
#df['clean_text'] = df['clean_text'].astype('str')

In [15]:
df_clean = df[['Target','clean_text']]

In [16]:
df_clean.reset_index(drop = True, inplace=True)

In [17]:
from gensim.models import Word2Vec

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(df_clean['clean_text'], df_clean['Target'], shuffle = True, test_size = 0.2)

In [20]:
import gensim.downloader as api

In [21]:
word2vec_transfer = api.load('glove-wiki-gigaword-100')

[=================================-----------------] 67.6% 86.5/128.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 128.1/128.1MB downloaded


In [22]:
word2vec_transfer['dog']

array([ 0.30817  ,  0.30938  ,  0.52803  , -0.92543  , -0.73671  ,
        0.63475  ,  0.44197  ,  0.10262  , -0.09142  , -0.56607  ,
       -0.5327   ,  0.2013   ,  0.7704   , -0.13983  ,  0.13727  ,
        1.1128   ,  0.89301  , -0.17869  , -0.0019722,  0.57289  ,
        0.59479  ,  0.50428  , -0.28991  , -1.3491   ,  0.42756  ,
        1.2748   , -1.1613   , -0.41084  ,  0.042804 ,  0.54866  ,
        0.18897  ,  0.3759   ,  0.58035  ,  0.66975  ,  0.81156  ,
        0.93864  , -0.51005  , -0.070079 ,  0.82819  , -0.35346  ,
        0.21086  , -0.24412  , -0.16554  , -0.78358  , -0.48482  ,
        0.38968  , -0.86356  , -0.016391 ,  0.31984  , -0.49246  ,
       -0.069363 ,  0.018869 , -0.098286 ,  1.3126   , -0.12116  ,
       -1.2399   , -0.091429 ,  0.35294  ,  0.64645  ,  0.089642 ,
        0.70294  ,  1.1244   ,  0.38639  ,  0.52084  ,  0.98787  ,
        0.79952  , -0.34625  ,  0.14095  ,  0.80167  ,  0.20987  ,
       -0.86007  , -0.15308  ,  0.074523 ,  0.40816  ,  0.0192

In [23]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

In [24]:
X_train_pad = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

In [25]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

In [26]:
def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(30, activation='tanh', return_sequences=True))
    model.add(layers.LSTM(20, activation='tanh', return_sequences=True))
    model.add(layers.LSTM(10, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model

In [27]:
model = init_model()

2022-12-02 11:07:42.864508: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-02 11:07:42.868171: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-02 11:07:42.868908: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Dimitris_Laptop): /proc/driver/nvidia/version does not exist
2022-12-02 11:07:42.890886: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [29]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss')

model.fit(X_train_pad, y_train, 
          batch_size = 32,
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/100
74/74 [==============================] - 21s 279ms/step - loss: 0.5305 - accuracy: 0.7486 - val_loss: 0.5580 - val_accuracy: 0.7406
Epoch 2/100
74/74 [==============================] - 20s 275ms/step - loss: 0.4883 - accuracy: 0.7796 - val_loss: 0.4867 - val_accuracy: 0.7663
Epoch 3/100
74/74 [==============================] - 22s 292ms/step - loss: 0.4523 - accuracy: 0.8081 - val_loss: 0.4595 - val_accuracy: 0.7980
Epoch 4/100
74/74 [==============================] - 21s 286ms/step - loss: 0.4134 - accuracy: 0.8229 - val_loss: 0.4278 - val_accuracy: 0.8188
Epoch 5/100
74/74 [==============================] - 21s 284ms/step - loss: 0.3766 - accuracy: 0.8459 - val_loss: 0.4282 - val_accuracy: 0.8257
Epoch 6/100
74/74 [==============================] - 21s 278ms/step - loss: 0.3504 - accuracy: 0.8552 - val_loss: 0.4709 - val_accuracy: 0.8000
Epoch 7/100
74/74 [==============================] - 21s 288ms/step - loss: 0.3295 - accuracy: 0.8730 - val_loss: 0.4376 - val_accuracy:

In [30]:
model.evaluate(X_test_pad, y_test)

27/27 [==============================] - 2s 70ms/step - loss: 0.4621 - accuracy: 0.8147


[0.46214932203292847, 0.8147268295288086]